# 手撕决策树

这篇文档最大的特点就是把笔者的思考过程写了出来，希望对于初学者如何去组织自己的思路，一步一步实现一个相对复杂的任务有所帮助。毕竟完成授人以鱼，不如授人以渔。  
需要提示的是：本文有些细节为了讲述的便利，假定了输入数据中使用了某些特定的变量名称，例如假设目标变量的字段名称为class。这牺牲了一定的通用性。大家可根据自己的需要来进行改造。

## 数据处理

In [232]:
import pandas as pd
data = pd.read_csv("/Users/lousiyuan/dataScience/gitSpace/DataScienceProject/Data/watermelon3_0.csv",header=None)  #这一句要替换为自己电脑上的文件路径
data.columns = ["index","color","root","noise","texture","navel","touch","density","sugar","class"]

In [233]:
data[:3]

,index,color,root,noise,texture,navel,touch,density,sugar,class
0,1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,是
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
2,3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是


还记得吗？第一个索引列其实是不需要的，这次我们用一个以前没有用过的操作来舍弃该列

In [234]:
data = data.drop(['index'],axis=1)  #axis=1表示要删除列
data[:3]

,color,root,noise,texture,navel,touch,density,sugar,class
0,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,是
1,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
2,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是


# 开始手动写一个决策树吧！

啥？头绪很乱，想不清楚对吧？那就慢慢来呗！ 

程序编写的重点之一就是把一个大的想法不断拆分成若干小的模块，最后加以整合。这样一个一个小的模块，可以是函数、类、包等不同的形式。其实自定义一个一个函数，合并成一个大的文件，就是最简单的形式。

首先，我在脑海里把决策树的执行过程想了一遍。然后，我意识到，里面其实是一个迭代、反复地分枝的过程。所以，第一步我想要实现一个分枝选择的功能。

那么，有哪些可供选择的分枝点呢？我们的数据有很多字段（列），每一个字段就是一个可以拿来进行分枝的节点，但一个字段中会有很多的值，在哪些值上进行分叉呢？这里面就有点门道了。我们知道决策树的核心就是分枝选择，不同的牛人提出了不同的分枝选择依据，所以才会有了ID3、C4.5、CART等不同的策略。这里我们以信息熵方法做为演示。

每一个字段是一个可供选择的分枝对象，里面又有不同的数值可做为分枝的判断条件，这样一个数据的需求，让我们想到了python中的字典(dict）数据结构。每个字段可以做为KEY，字段里可做为分枝选择条件的值用一个列表（list)来保存。

In [235]:
from collections import defaultdict
def get_possible_branch(data):
    possible_branch = defaultdict(list)
    #遍历所有字段，得到每个字段可能的分裂点
    #注意最后一列是目标变量，不能做为分裂选择的变量
    for col in data.drop(["class"],axis=1).columns:
        #假装我们已经有了一个函数可以用来返回每个字段上的可能值
        possible_branch[col] = get_possible_value(data,str(col))   
    return possible_branch

这时我们就意识到：哦，还需要一个获得每个字段可能做为分裂条件值的函数。

In [236]:
def get_possible_value(data,col):
    possible_value = []
    # 呃，怎么办，好像连续型字段与离散型字段的处理方法不一样唉
    return possible_value

事实上，单纯依靠数据中呈现的数值本身，并不能准备断定一个字段是连续的还是离散的，比如一个表示年龄的字段，就是一个一个有序数值；一个数值为0，1，2的字段，也有可能只是某一个离散变量。所以我们可以默认把字段当作连续型的，这也是很多决策树算法包的处理方式。其实，这也是为什么我们需要对离散变量进行one-hot-encoding处理的原因所在嘛！

当然，有些最近的算法包已经可以直接处理类别型变量了，例如catboost，这是一种集成学习算法。感兴趣的同学可以关注本人微信公众号：《实战数据分析挖掘》（funCodingFunLife） ，上面有关于catboost原理、编码的实战讲解。

那连续变量，怎么得到一系列分裂点呢？通常的做法是对变量值先去重，再进行排序，然后每相邻两个值取平均数，生成一个分裂的数值，如此得到一系列值。  
So, 我们可以继续写上面的函数了。

In [237]:
import numpy as np
def get_possible_value(data,col):
    possible_value = []
    unique_values = np.unique(data[col])   #numpy.unique本身返回的就已经是排序后的值，所以不再需要单独排序
    if len(unique_values) == 2:
        potential_split = (unique_values[0] + unique_values[1]) / 2
        possible_value.append(potential_split)
    else:
        for index in range(1,len(unique_values)):   #细心的同学可以思考下这里为什么从下标1开始迭代？
            current_value = unique_values[index]
            previous_value = unique_values[index - 1]
            potential_split = (current_value + previous_value) / 2
            possible_value.append(potential_split)
    return possible_value

到这里，我们已经写了3个自定义函数了，要及时测试一下是否功能表现如我们所预期。注意这里仅仅是测试，后面还会正式进行数据预处理。  
要记得先对类别型变量进行one-hot-encoding处理哦！

In [238]:
#对特征变量进行 ONE-HOT-ENCODING
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categories="auto",handle_unknown = 'ignore')

#对类别型的变量进行 ONE-HOT-ENCODING
data_cate = data[["color","root","noise","texture","navel","touch"]]
ohe = ohe.fit(data_cate)
data_cate_ohe = pd.DataFrame(ohe.transform(data_cate).toarray())
data_cate_ohe.columns = ohe.get_feature_names()

data_continuous = pd.DataFrame(data[["density","sugar"]]).reset_index(drop=True)  #注意这里对索引进行重新排序，否则后面合并的时候会出现错位

data_label = pd.DataFrame(data["class"].map({"是":1,"否":0})).reset_index(drop=True) 

#再跟原来的特征变量合并到一起
data_concat = pd.concat([data_cate_ohe,data_continuous,data_label],axis=1)
data_concat[:2]

,x0_乌黑,x0_浅白,x0_青绿,x1_硬挺,x1_稍蜷,x1_蜷缩,x2_沉闷,x2_浊响,x2_清脆,x3_模糊,x3_清晰,x3_稍糊,x4_凹陷,x4_平坦,x4_稍凹,x5_硬滑,x5_软粘,density,sugar,class
0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.697,0.460,1
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.774,0.376,1


In [239]:
get_possible_branch(data_concat)

defaultdict(list,
            {'x0_乌黑': [0.5],
             'x0_浅白': [0.5],
             'x0_青绿': [0.5],
             'x1_硬挺': [0.5],
             'x1_稍蜷': [0.5],
             'x1_蜷缩': [0.5],
             'x2_沉闷': [0.5],
             'x2_浊响': [0.5],
             'x2_清脆': [0.5],
             'x3_模糊': [0.5],
             'x3_清晰': [0.5],
             'x3_稍糊': [0.5],
             'x4_凹陷': [0.5],
             'x4_平坦': [0.5],
             'x4_稍凹': [0.5],
             'x5_硬滑': [0.5],
             'x5_软粘': [0.5],
             'density': [0.244,
              0.294,
              0.3515,
              0.38149999999999995,
              0.42000000000000004,
              0.4590000000000001,
              0.5185000000000001,
              0.5745,
              0.6005,
              0.621,
              0.6365000000000001,
              0.648,
              0.6615,
              0.6815,
              0.7080000000000001,
              0.7465],
             'sugar': [0.0495,
              0.074,
   

现在，我们已经知道了有哪些可以分裂的节点，每个节点上有哪些候选数值可以作为判断的依据。那么问题来了？怎么选择最优的分裂点？   
回想一下，我们上文说本文用信息熵来做为判断依据。所以我们还需要创建一个计算信息熵的函数。

In [240]:
def calculate_entropy(data):
    #取最后一列标签值
    label_column = data.iloc[:,-1]
    _, counts = np.unique(label_column, return_counts=True) #这个操作高级了！ 一下子同时得到枚举值列表，以及各枚举值的数量； 这里counts其实是一个列表
    probabilities = counts / counts.sum()   #这里probabilities仍然是一个列表
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

In [241]:
calculate_entropy(data_concat)

0.9975025463691152

有了计算熵的函数后，我们就可以基于此来计算分枝前后得到的信息增益，然后横向对比所有可能的分裂点和分裂值，找出信息增益最大的那个做为最终的分裂点和值。  
分枝后，原来的数据集被拆成了左枝、右枝，新的信息熵是两个集合的加权和，权重是各自样本量占原来总样本量的比例。  
回顾了这些基础后，我们就可以动手来做了。

In [242]:

def calculate_overall_entropy(data_below, data_above):
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n
    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    return overall_entropy


#定义一个分枝的函数，根据某个特征变量的某个值，将原数据集划分成左枝一个数据集、右枝一个数据集
def split_data(data, split_column, split_value):
    split_column_values = data.loc[:, str(split_column)]
    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    return data_below, data_above


def determine_best_split(data, potential_splits):
    overall_entropy = 9999
    for column_index in potential_splits:    #还记得可行分裂点的格式吗？是个字典，key是特征变量，value是一系列可行的分裂点构成的列表list
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    return best_split_column, best_split_value

In [243]:
potential_splits = get_possible_branch(data_concat)
a,b=determine_best_split(data_concat,potential_splits)
a,b

('sugar', 0.126)

In [244]:
c,d = split_data(data_concat,a,b)
len(c),len(d)

(5, 12)

现在，我们在脑海中想一想怎么不断地分枝下去？每一次决定是否要分枝时，其实需要看看当前数据的目标变量是不是“纯”的————只有某一个类了，如果是，那就不需要继续分下去了。所以这里我们需要一个函数来检查数据的纯度。

In [245]:
def check_purity(data):
    label_column = data.iloc[:, -1]
    unique_classes = np.unique(label_column)
    if len(unique_classes) == 1:
        return True
    else:
        return False

In [246]:
check_purity(data_concat)

False

继续。  
分了一个新枝后，在这个新枝上的样本集，怎么来训练得出其目标变量呢？ 如果已经只剩一个目标变量值了，那显然将该数据集上所有样本均预测为该值即可。但如果还有多个目标值呢？即当前的数据并不“纯”，那就少数服从多数进行投票吧！

In [247]:
def classify_data(data):
    label_column = data.iloc[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)
    index = counts_unique_classes.argmax() #得到频数最大值的下标
    classification = unique_classes[index]
    return classification

In [248]:
def decision_tree_algorithm(data, counter=0):
    #如果数据已经仅有一个分类了，说明已经是纯的数据了，就不再需要继续分枝
    if check_purity(data):   
        classification = classify_data(data)
        return classification
    else:    
        counter += 1
        # helper functions 
        potential_splits = get_possible_branch(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        # instantiate sub-tree
        question = "{} <= {}".format(split_column, split_value)
        sub_tree = {question: []}
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter)
        no_answer = decision_tree_algorithm(data_above, counter)
        sub_tree[question].append(yes_answer)
        sub_tree[question].append(no_answer)
        return sub_tree

In [249]:
tree = decision_tree_algorithm(data_concat,0)
from pprint import pprint
pprint(tree)

{'sugar <= 0.126': [0,
                    {'x1_蜷缩 <= 0.5': [{'x4_稍凹 <= 0.5': [0,
                                                        {'sugar <= 0.3035': [1,
                                                                             0]}]},
                                      1]}]}


事实上，一棵决策树模型经过训练后，留下来的是什么呢？就是每一步使用哪个特征变量的哪个值来进行分裂。然后面对未知的数据，根据这些规则来执行就是了。但使用什么样的数据结构来保存这些规则，确实值得好好思考。

In [250]:
list(tree.keys())[0]

'sugar <= 0.126'

In [251]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split()
    # ask question
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

### 来吧，训练+预测+评估来一把

#### 准备数据

从上文很多自定义函数的逻辑中，估计大家也猜到了。本文中我们拆分数据时，需要将特征变量与目标变量放在一起。所以我们先不用sklearn中自带的train_test_split。而是自己编写一个。

In [252]:
import random
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [253]:
train_df , test_df = train_test_split(data,0.2)
len(train_df),len(test_df)

(14, 3)

In [254]:
#对特征变量进行 ONE-HOT-ENCODING
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categories="auto",handle_unknown = 'ignore')

#对类别型的变量进行 ONE-HOT-ENCODING
train_cate = train_df[["color","root","noise","texture","navel","touch"]]
ohe = ohe.fit(train_cate)
train_cate_ohe = pd.DataFrame(ohe.transform(train_cate).toarray())
train_cate_ohe.columns = ohe.get_feature_names()

train_continuous = pd.DataFrame(train_df[["density","sugar"]]).reset_index(drop=True)  #注意这里对索引进行重新排序，否则后面合并的时候会出现错位

train_label = pd.DataFrame(train_df["class"].map({"是":1,"否":0})).reset_index(drop=True) 

#再跟原来的特征变量合并到一起
train_concat = pd.concat([train_cate_ohe,train_continuous,train_label],axis=1)
# train_concat[:2]
len(train_concat)

14

#### 用训练数据来建立一棵决策树

In [255]:
len(train_concat),len(data)

(14, 17)

In [256]:
tree = decision_tree_algorithm(train_concat)

In [257]:
pprint(tree)

{'sugar <= 0.20650000000000002': [0,
                                  {'density <= 0.38149999999999995': [0, 1]}]}


#### 预测

照例先对测试数据也进行预处理

In [258]:
# 先对测试数据也进行一些数据预处理，注意要直接使用那些用于处理训练数据的转换操作实例，可以直接调用在训练数据fit好的算子
test_cate = test_df[["color","root","noise","texture","navel","touch"]]
#ohe是上面已经在训练数据上fit后的算子，这里可以直接调用
test_cate_ohe = pd.DataFrame(ohe.transform(test_cate).toarray())
test_cate_ohe.columns = ohe.get_feature_names()

test_continous = pd.DataFrame(test_df[["density","sugar"]]).reset_index(drop=True)

#对标签变量进行0-1编码，对于二分类，可以使用LabelBinarizer，但笔者更喜欢直接用map来显示转换
test_label = pd.DataFrame(test_df["class"].map({"是":1,"否":0})).reset_index(drop=True) 

#合并
test_concat = pd.concat([test_cate_ohe,test_continous,test_label],axis=1)
len(test_concat)


3

In [259]:
y_predict = test_concat.apply(classify_example, axis=1, args=(tree,))

### 评估准确率

In [261]:
from sklearn.metrics import accuracy_score
y_real = test_concat["class"]
accuracy_score(y_predict,y_real)

0.6666666666666666

到这里，我们基本实现了一个无剪枝逻辑的决策树。

# THE END